Use the "transform_all_images_of_test_dataset.py" script inside ComfyUI to generate images for the test dataset

In [22]:
import PIL.ImageOps
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from PIL import ImageChops
embedding_name = ("fabhsr")

transformed_images_folder = "embedding_datasets/" + embedding_name + "/results/transform-all/"
dataset_folder = "datasets/deep-fashion/In-shop Clothes Retrieval Benchmark/Img/"
df = pd.read_csv("datasets/query_list.csv")


def grand_sum(image: Image):
    return np.sum(np.array(image) == True)

with open(transformed_images_folder + "/mask_overlap.csv", "w") as csv:
    for image_path in tqdm(df["image_name"]):
        mask_path = (".".join(image_path.split(".")[:-1])) + "_mask_sam.png"
        gt_mask_path = (".".join(image_path.split(".")[:-1])) + "_mask.png"

        M_transformed_path = transformed_images_folder + mask_path
        M_groundtruth_path = dataset_folder + gt_mask_path
        M_original_path = dataset_folder + mask_path

        M_groundtruth = Image.open(M_groundtruth_path).convert("1")
        M_transformed = PIL.ImageOps.invert(Image.open(M_transformed_path).convert("1"))
        M_original = PIL.ImageOps.invert(Image.open(M_original_path).convert("1"))

        # due to vae encoding image dimensions are rounded to be divisible by 8
        # possibly introduces some pixel shift worsening this metric by a very small percentage
        M_transformed = M_transformed.resize(M_original.size)

        M_original_filled =  ImageChops.logical_or(M_original,M_groundtruth)
        M_transformed_filled = ImageChops.logical_or(M_transformed,M_groundtruth)

        M_overlap =  ImageChops.subtract(M_transformed_filled,M_original_filled)
        percentage_overlap = grand_sum(M_overlap) / grand_sum(M_original_filled)
        csv.write(image_path + "," + str(percentage_overlap) + "\n")

100%|██████████| 416/416 [00:10<00:00, 39.27it/s]
